# 🧮 Fairness Analysis – Intermediate Level

This notebook explores fairness across machine learning models using two datasets:
- `loan_data.csv`: Loan approval dataset with injected caste and religion bias.
- `promotion_data.csv`: Account promotion dataset with exposure bias.

We compare models **with and without protected attributes**, and compute basic group fairness metrics like **Disparate Impact Ratio**.

### 🛡️ Protected Attributes
- `gender`
- `caste`
- `religion`
- `accent_score`

### 🧮 Neutral Attributes
- `age`
- `income`
- `credit_history_score`
- `number_of_loans`
- `savings_balance`
- `years_with_bank`

## 🧪 Intermediate Challenge Instructions

In this level, your goal is to explore **group-level disparities** in both **loan approval** and **promotion exposure**, combining EDA and fairness-aware ML modeling.

---

### 📘 About the Notebook

We have included an example Jupyter notebook (`intermediate_fairness_analysis.ipynb`) to help you get started.  
Use it as a **template** — feel free to build on it, expand the analysis, and apply the same techniques to the promotion dataset.

---

### 🔍 1. Exploratory Data Analysis (EDA)

Begin with data understanding:
- Explore distributions of **protected attributes**: `caste`, `religion`, `gender`
- Examine **neutral features**: `credit_history_score`, `number_of_loans`, `savings_balance`, etc.
- Visualize group-level statistics using:
  - Bar plots
  - Countplots
  - Correlation heatmaps

✅ Tip: Replicate or build on the visualizations provided in the Beginner notebook to support your fairness analysis.

---

### 🤖 2. Train and Compare ML Models (Loan Dataset)

Use the `loan_df` to predict the `approved` outcome.

Train **two models**:
- **With protected attributes** (e.g., `caste`, `religion`, `gender`)
- **Without protected attributes** (only neutral features)

Suggested models:
- Logistic Regression
- Decision Tree Classifier

Evaluate model performance using:
- Accuracy
- Confusion Matrix

⚠️ Note: This is a synthetic dataset, so perfect accuracy may indicate it's too simple or that bias is encoded in correlated non-protected features.

---

### 📊 3. Fairness Metrics (Loan + Promotion)

Calculate **group fairness metrics** for both `loan_df` and `promo_df`:

#### For Loan Dataset:
- **Disparate Impact Ratio (DIR)**: Compare approval rates between groups (e.g., SC/ST vs General)
- **Equal Opportunity Difference**: Difference in true positive rates across groups
- **Accuracy by Group**

#### For Promotion Dataset:
- Treat `assigned_promotion != "No_Promo"` as the positive class (received benefit)
- Repeat similar fairness metrics (DIR, accuracy, etc.) by `caste` or `religion`

---

### ✅ Final Task

Summarize your findings:
- Do protected groups receive systematically different outcomes?
- Does removing protected features improve fairness?
- How does fairness vary between **loan approval** and **promotion exposure** tasks?

You are encouraged to structure your analysis with clear sections and interpretations.


In [5]:
import pandas as pd
import numpy as np

# Load loan data
loan_df = pd.read_csv("data/loan_data.csv")
promo_df = pd.read_csv('./data/promotion_data.csv')


# Preview
loan_df.head()

,user_id,gender,caste,religion,age,income,accent_score,credit_history_score,number_of_loans,savings_balance,years_with_bank,loan_score,approved
0,U000,Male,SC/ST,Other,41,65247,54.84,739.0,2,47828,7,65.97,1
1,U001,Female,OBC,Hindu,52,92752,96.98,707.6,2,48924,17,55.00,0
2,U002,Male,SC/ST,Hindu,43,86573,77.37,638.4,0,75182,7,71.31,1
3,U003,Male,SC/ST,Hindu,53,89101,63.78,709.5,1,94065,14,89.54,1
4,U004,Male,OBC,Other,23,56646,64.51,770.0,2,60052,15,71.09,1


## ⚙️ Encode categorical features and define X/y

In [3]:
# Encode categorical variables
loan_df_encoded = pd.get_dummies(loan_df.drop(columns=['user_id']), drop_first=True)

# Target variable
y = loan_df_encoded['approved']
X = loan_df_encoded.drop(columns=['approved'])

# Identify protected attribute columns
protected_cols = [col for col in X.columns if any(p in col for p in ['gender_', 'caste_', 'religion_', 'accent_score'])]

# Neutral-only feature set
X_neutral = X.drop(columns=protected_cols)


## 🤖 Logistic Regression and Decision Tree Models

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)
Xn_train, Xn_test = train_test_split(X_neutral, test_size=0.2, stratify=y, random_state=42)

# Logistic Regression with full features
lr_full = LogisticRegression(max_iter=1000)
lr_full.fit(X_train, y_train)
acc_lr_full = accuracy_score(y_test, lr_full.predict(X_test))

# Logistic Regression without protected features
lr_fair = LogisticRegression(max_iter=1000)
lr_fair.fit(Xn_train, y_train)
acc_lr_fair = accuracy_score(y_test, lr_fair.predict(Xn_test))

# Decision Tree with full features
dt_full = DecisionTreeClassifier(max_depth=5, random_state=42)
dt_full.fit(X_train, y_train)
acc_dt_full = accuracy_score(y_test, dt_full.predict(X_test))

# Decision Tree without protected features
dt_fair = DecisionTreeClassifier(max_depth=5, random_state=42)
dt_fair.fit(Xn_train, y_train)
acc_dt_fair = accuracy_score(y_test, dt_fair.predict(Xn_test))

# Print results
pd.DataFrame({
    'Model': ['LogReg w/ Prot', 'LogReg w/o Prot', 'Tree w/ Prot', 'Tree w/o Prot'],
    'Accuracy': [acc_lr_full, acc_lr_fair, acc_dt_full, acc_dt_fair]
})

/Users/deb/miniconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,Model,Accuracy
0,LogReg w/ Prot,1.00
1,LogReg w/o Prot,0.95
2,Tree w/ Prot,1.00
3,Tree w/o Prot,1.00


**Observations:** Since this is a small synthetic dataset, the models achieve high accuracy even without protected attributes. This suggests either the dataset is too simple or the bias is encoded in correlated non-protected features. **It is important to investigate further whether the models are learning genuine patterns or replicating existing biases.**

This comparison allows us to evaluate whether using protected attributes **increases model performance** — and whether **removing them improves fairness**.

> 🔎 Next Steps: Implement group-level fairness metrics such as:
> - **Disparate Impact Ratio**
> - **Equal Opportunity Difference**
> - **Accuracy by Group**

## 📐 Fairness Metrics
We compute three metrics using caste as the protected attribute:

### 1. **Disparate Impact Ratio (DIR)**
Let $P_+^{adv}$ be the favorable outcome rate for the advantaged group, and $P_+^{dis}$ for the disadvantaged group:

$$ DIR = \frac{P_+^{dis}}{P_+^{adv}} $$

### 2. **Equal Opportunity Difference (EOD)**
True positive rate (TPR) for each group:

$$ EOD = TPR_{dis} - TPR_{adv} $$

### 3. **Group Accuracy**
Let $Acc_g$ be accuracy for group $g$:

$$ Acc_g = \frac{TP + TN}{TP + TN + FP + FN} $$


In [6]:
def compute_dir_robust(df, protected_col, outcome_col, adv_value, dis_value):
    # Normalize values to lowercase and strip whitespace
    df = df.copy()
    df[protected_col] = df[protected_col].str.strip().str.lower()
    adv_value = adv_value.strip().lower()
    dis_value = dis_value.strip().lower()

    # Compute mean outcome for advantaged and disadvantaged groups
    p_adv = df[df[protected_col] == adv_value][outcome_col].mean()
    p_dis = df[df[protected_col] == dis_value][outcome_col].mean()

    # Return ratio with handling for zero or NaN
    if pd.isna(p_adv) or p_adv == 0:
        return None  # or return np.nan or raise a warning
    return round(p_dis / p_adv, 3)


In [7]:
compute_dir_robust(loan_df, 'caste', 'approved', 'General', 'SC/ST')


0.688

**Observations:** This means that individuals from the SC/ST caste group are approved for loans at only 68.8% the rate of individuals from the General caste group.

**Interpretation:** A value below 0.80 is commonly considered a threshold for potential disparate impact (known as the "four-fifths rule"). At 0.688, this example is below the threshold, indicating a possible bias that deserves further analysis.